### Acknowledgement
Name: Zuha Aqib     
ERP ID: 26106    
Section: 10am Miss Solat     
Date: (written on) 18-Feb-25     

code has been taken from Miss Solat's code files and written by Zuha themselves. 

In [2]:
from datetime import datetime
print("Last time code executed: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Last time code executed:  2025-02-18 20:09:48


# Imports
here we import all the necessary libraries and modules

In [ ]:
# to load the dataset
import warnings
warnings.filterwarnings('ignore')

# to handle the data and perform numerical operations on it
import pandas as pd
import numpy as np

# to do natural language processing
import nltk

# preprocessing: words tokenization
from nltk.tokenize import word_tokenize

# preprocessing: stop word removal
from nltk.corpus import stopwords

# preprocessing: stemmming and lemmatization
from nltk.stem import PorterStemmer, WordNetLemmatizer

# vectorization: BOW
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# vectorization: LSA
from sklearn.decomposition import TruncatedSVD

# clustering: k means
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# to plot the graph
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Load the Dataset
Here we load the dataset from a csv file and then save it into a variable

In [18]:
with open('news_Feb_14.csv','r',encoding="utf8") as f:
    documents = f.readlines()
f.close()

documents

['\ufefftitle,date\n',
 'ECC endorses purchase of $582mn capital shares in BRICS’s New Development Bank,14/02/2025\n',
 '"Netanyahu’s statement to establish Palestinian state in Saudi Arabia irresponsible, provocative: FO",14/02/2025\n',
 '"India, US agree to resolve trade and tariff rows after Trump-Modi talks",14/02/2025\n',
 'Aurangzeb discusses Pakistan’s structural reforms with IFC chief,14/02/2025\n',
 '"HBL, S&P Global launch Pakistan’s first manufacturing PMI",14/02/2025\n',
 'KSE-100 closes 479 points lower as late-session selling erases intra-day gains,14/02/2025\n',
 'PPL executes agreements for Sui Gas Field operations,14/02/2025\n',
 '"Hubco Green, PSO collaborate to setup EV charging infrastructure across Pakistan",14/02/2025\n',
 'Rupee records marginal improvement against US dollar,14/02/2025\n',
 '"At least 9 dead, 7 injured in Balochistan’s Harnai blast",14/02/2025\n',
 '"Gold adds to record rally, reaches Rs306,200 per tola in Pakistan",14/02/2025\n',
 '"IMF’s 2024 A

# Preprocessing
here we perform the preprocessing of data like:
- lowercase text
- stopword removal
- stemming or lemmatization
- n-grams

## Lowercase Conversion
here we convert our text to lowercase to have accurate DTM/TDM

In [8]:
documents = [document.lower() for document in documents]

documents

['\ufefftitle,date\n',
 'ecc endorses purchase of $582mn capital shares in brics’s new development bank,14/02/2025\n',
 '"netanyahu’s statement to establish palestinian state in saudi arabia irresponsible, provocative: fo",14/02/2025\n',
 '"india, us agree to resolve trade and tariff rows after trump-modi talks",14/02/2025\n',
 'aurangzeb discusses pakistan’s structural reforms with ifc chief,14/02/2025\n',
 '"hbl, s&p global launch pakistan’s first manufacturing pmi",14/02/2025\n',
 'kse-100 closes 479 points lower as late-session selling erases intra-day gains,14/02/2025\n',
 'ppl executes agreements for sui gas field operations,14/02/2025\n',
 '"hubco green, pso collaborate to setup ev charging infrastructure across pakistan",14/02/2025\n',
 'rupee records marginal improvement against us dollar,14/02/2025\n',
 '"at least 9 dead, 7 injured in balochistan’s harnai blast",14/02/2025\n',
 '"gold adds to record rally, reaches rs306,200 per tola in pakistan",14/02/2025\n',
 '"imf’s 2024 a

## Stop Words Removal
here we remove stop words. but we initialize stop words in 

In [16]:
stop_word = True
stop_word_language = 'english'